In [ ]:
!pip install moviepy --quiet

In [ ]:
!pip install pytube ffmpeg-python --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
!pip install SpeechRecognition --quiet
!pip install pydub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 26.7 MB/s eta 0:00:00


In [ ]:
#!pip install Office365-REST-Python-Client

In [ ]:
#!pip install requests_ntlm

In [ ]:
#!pip uninstall ffmpeg ffmpeg-python

In [ ]:
from pytube import YouTube
import ffmpeg
import requests
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import json
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.utils import make_chunks
import os

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def is_mp4_playable(file_path):
  try:
      # Probe the file to get information about it
      probe = ffmpeg.probe(file_path)
      # Check if the file format is recognized as video
      if 'streams' in probe and any(stream['codec_type'] == 'video' for stream in probe['streams']):
          print("MP4 file is playable.")
          return True
      else:
          print("MP4 file is corrupt or non-playable.")
          return False
  except ffmpeg.Error as e:
      #print("Error occurred:", e.stderr)
      return False

In [ ]:
def extract_audio(link, output_file_path):

  if urlparse(link).netloc == "www.youtube.com":
    yt = YouTube(link)
    video_path = yt.streams[0].url

  elif urlparse(link).netloc == "www.linkedin.com":
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    data_linkedin = json.loads(soup.find('script', type='application/ld+json').text)
    if data_linkedin['isAccessibleForFree'] == True:
      video_path = json.loads(soup.video['data-sources'])[0]['src']
    else:
      print("Sorry! Can't extract audio. Please make sure the video is free for access.")
      return

  elif urlparse(link).netloc == "www.coursera.org":
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    data_coursera = json.loads(soup.find('script', type='application/ld+json').text)
    video_path = data_coursera['@graph'][1]['contentURL']

  elif urlparse(link).netloc not in ["www.youtube.com", "www.linkedin.com", "www.coursera.org"]:
    print("Sorry! Can't extract audio. Please make sure the video link is valid.")
    return


  else:
    if(is_mp4_playable(link)):
      video_path = link
    else:
      print("Sorry! Can't extract audio. Please make sure the video file exists and is not corrupted.")
      return


  audio, err = (
      ffmpeg
      .input(video_path)
      .output("pipe:", format='mp3', acodec='libmp3lame', audio_bitrate='320k')
      .run(capture_stdout=True)
  )


  with open(output_file_path, 'wb') as f:
      f.write(audio)

  print("Audio extraction complete.")

In [ ]:
# Invalid URL
extract_audio("https://exampleurl",'/content/drive/My Drive/LLM_Project/link_audio.mp3')

Sorry! Can't extract audio. Please make sure the video link is valid.


In [ ]:
# Youtube
extract_audio("https://www.youtube.com/watch?v=AM8D4j9KoaU&pp=ygUVZGV0ciBvYmplY3QgZGV0ZWN0aW9u",'/content/sample_data/youtube_audio.mp3')

Audio extraction complete.


In [ ]:
# Linkedin free course
extract_audio("https://www.linkedin.com/learning/administrative-professional-weekly-tips/managing-morale-in-your-office?&u=74653650",'/content/sample_data/linkedin_audio_free.mp3')

Audio extraction complete.


In [ ]:
# Linkedin non-free course
extract_audio("https://www.linkedin.com/learning/tableau-for-data-scientists/solution-connecting-to-data?resume=false&u=74653650",'/content/drive/My Drive/LLM_Project/linkedin_audio_paid.mp3')

Sorry! Can't extract audio. Please make sure the video is free for access.


In [ ]:
# Coursera free course
extract_audio("https://www.coursera.org/learn/the-science-of-well-being/lecture/yaNZk/part-2-goal-setting",'/content/sample_data/coursera_audio_free.mp3')

Audio extraction complete.


In [ ]:
# Uploaded MP4 video file - correct
extract_audio("/content/drive/My Drive/LLM_Project/LLM_lecture.mp4",'content/sample_data/uploaded_MP4_audio.mp3')

Sorry! Can't extract audio. Please make sure the video link is valid.


In [ ]:
# Uploaded MP4 video file - path doesn't exist
extract_audio("/content/drive/My Drive/LLM_Project/LLM_lecture.mp3",'/content/drive/My Drive/LLM_Project/uploaded_MP4_audio.mp3')

Sorry! Can't extract audio. Please make sure the video file exists and is not corrupted.


In [ ]:
# Uploaded MP4 video file - corrupt/non-playable file
extract_audio("/content/drive/My Drive/LLM_Project/empty_video.mp4",'/content/drive/My Drive/LLM_Project/uploaded_MP4_audio.mp3')

Sorry! Can't extract audio. Please make sure the video file exists and is not corrupted.


In [ ]:
urlparse("https://exampleurl.uyg").netloc

'exampleurl.uyg'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 23.0 MB/s eta 0:00:00


In [ ]:
def divide_audio_chunks(audio_file, chunksize = 30000):
    mp3_audio = AudioSegment.from_mp3(audio_file)

    # Split the audio into chunks
    chunks = make_chunks(mp3_audio, chunksize)

    return chunks


In [ ]:
def process_chunks(audio_chunks):
  whole_speech = ""
  recognizer = sr.Recognizer()
  for i, chunk in enumerate(audio_chunks):
    audio = chunk.export(format="wav")
    with sr.AudioFile(audio) as source:
      audio_data = recognizer.record(source)
      try:
        text = recognizer.recognize_google(audio_data)
        print(f"Chunk {i+1}: {text}")
        whole_speech += text
      except sr.UnknownValueError:
        print(f"Chunk {i+1}, Google Speech Recognition could not understand audio")
      except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
  return whole_speech

In [ ]:
sound_file = "/content/sample_data/youtube_audio.mp3"

In [ ]:
audio_chunks = divide_audio_chunks(sound_file)

In [ ]:
speech = process_chunks(audio_chunks)
print("\n\n Entire Speech")
print(speech)

Chunk 1: I need to learn Transformers I have this kind of imposter syndrome crisis recently don't get me wrong I feel pretty comfortable working with almost any computer vision algorithm that is popular right now but when it comes to Transformers I don't feel comfortable enough that's why I decided that this week will dive deeper in the world of object detection Transformers and we will learn how to train one of them on
Chunk 2: play some data set so if you are just like me and you would like to learn something new today sit back relax make some coffee and prepare yourself for 25 minutes of pure struggle but I promise that we will use plenty of super cool and useful tools today like by torch lightning Transformers supervision tensorboard so there's a lot of unpack let's not waste any more time and train some models and as usual I prepared for you a
Chunk 3: locate The jupyter Notebook you will find it in the description below so you can follow my steps during the tutorial we scroll thr

In [ ]:
sound_file = "/content/sample_data/linkedin_audio_free.mp3"

In [ ]:
audio_chunks = divide_audio_chunks(sound_file)
speech = process_chunks(audio_chunks)
print("\n\n Entire Speech")
print(speech)

Chunk 1: how is the morale in your office is everyone glad to work there what is your contribution to the energy that exists in your work environment I hope that you are one of the main reasons people love coming to work as a seasoned administrative support professional I have worked in a number of different offices over my 30-year career one of my defining moments was when I was in a volatile work environment
Chunk 2: however instead of getting sucked into it I realized it was my decision how I manage my morale and what I contributed I had just as much influence as others and setting a tone for the office and I chose to make it better not worse morale and our offices is mainly seen as the attitudes Outlook and satisfaction of the employees who work there I have found that of
Chunk 3: times were the morale is low or high is attributed to the leadership in that company it is very easy to point fingers outward and upward for morale is low however I don't subscribe to that philosophy I un

In [ ]:
sound_file = "/content/sample_data/coursera_audio_free.mp3"

In [ ]:
audio_chunks = divide_audio_chunks(sound_file)
speech = process_chunks(audio_chunks)
print("\n\n Entire Speech")
print(speech)

Chunk 1: the last thing will end on is one of the most empirically validated sets of things for Behavior change and that's thinking about your goals in a very specific way setting your goals using a couple techniques that can make your goal achievement a lot easier so we all kind of think we have goals like some of you may have goals for some of the stuff you mentioned but to actually actualize those you have to consider them as goals research suggests that thinking about your goals and very specific ways can let you achieve them even more
Chunk 2: and so a feel of the ways that you can think about them first start with the idea of figuring out what your goal is and so that is what I'm going to refer to as goal specificity and I mean something very specific haha about goal specificity it's this idea that you have some quantitative Precision about your goal like yes I'm going to meditate but how like where like am I going to meditate here in my house am I going to meditate by myself am 